In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=a2e1fbfc1102c73d176c98dec0163ef0cbe326157bf2bfd0e99ba24e2a3397db
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=0319f85031fb9b9cd77cdebbce5c41f5bf55c27f5282016ef2796a75db915493
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4192 - accuracy: 0.8496 - val_loss: 0.2917 - val_accuracy: 0.8935
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2766 - accuracy: 0.8997 - val_loss: 0.2879 - val_accuracy: 0.8983
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2224 - accuracy: 0.9172 - val_loss: 0.2569 - val_accuracy: 0.9040


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3302 - accuracy: 0.0999 - val_loss: 2.3035 - val_accuracy: 0.0942
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3037 - accuracy: 0.1004 - val_loss: 2.3037 - val_accuracy: 0.1008
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 2.3039 - accuracy: 0.1009 - val_loss: 2.3037 - val_accuracy: 0.1055


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4242 - accuracy: 0.8474 - val_loss: 0.3165 - val_accuracy: 0.8788
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2751 - accuracy: 0.8993 - val_loss: 0.2768 - val_accuracy: 0.8998
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2246 - accuracy: 0.9162 - val_loss: 0.2821 - val_accuracy: 0.9007


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4196 - accuracy: 0.8515 - val_loss: 0.3380 - val_accuracy: 0.8767
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2747 - accuracy: 0.9003 - val_loss: 0.2787 - val_accuracy: 0.8995
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.2191 - accuracy: 0.9191 - val_loss: 0.2507 - val_accuracy: 0.9102


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4539 - accuracy: 0.8361 - val_loss: 0.3864 - val_accuracy: 0.8635
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3325 - accuracy: 0.8779 - val_loss: 0.3487 - val_accuracy: 0.8738
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2852 - accuracy: 0.8933 - val_loss: 0.3596 - val_accuracy: 0.8743


INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                743456    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 773,146
Trainable params: 773,146
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1795 - accuracy: 0.9337 - val_loss: 0.2613 - val_accuracy: 0.9078
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1426 - accuracy: 0.9470 - val_loss: 0.3187 - val_accuracy: 0.9075
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1151 - accuracy: 0.9566 - val_loss: 0.2993 - val_accuracy: 0.9082
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0911 - accuracy: 0.9660 - val_loss: 0.3176 - val_accuracy: 0.9078
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0758 - accuracy: 0.9719 - val_loss: 0.3673 - val_accuracy: 0.9128
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0622 - accuracy: 0.9769 - val_loss: 0.4017 - val_accuracy: 0.9068
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0522 - accuracy: 0.9814 - val_loss: 0.4300 - val_accuracy